# 추천 시스템 (Recommendation System)
1. 콘텐츠 기반 필터링 (Content-based Filtering)
    - 아이템의 속성을 기반으로 사용자에게 적합한 아이템 추천
2. 협업 필터링 (Collaborative Filtering)
    - 사용자들 간의 유사성을 기반으로 추천
    - 사용자 기반과 아이템 기반으로 각각 추천할 수 있음
3. 하이브리드 추천 시스템 (Hybrid Recommendation System)
    - 협업 필터링과 콘텐츠 기반 필터링을 결합하여 추천

- 영화 데이터
    1. **id**: 영화의 고유 ID를 나타냄.
    2. **title**: 영화의 제목.
    3. **budget**: 영화 제작에 소요된 예산 (단위: USD).
    4. **popularity**: 영화의 인기 점수. TMDb에서 제공하는 영화의 인기도를 나타냄.
    5. **genres**: 영화의 장르를 나타내며, 여러 장르가 포함된 경우 리스트로 표현됨.
    6. **overview**: 영화의 줄거리나 개요를 설명하는 텍스트.
    7. **release_date**: 영화의 개봉 날짜.
    8. **revenue**: 영화의 총 수익 (단위: USD).
    9. **runtime**: 영화의 상영 시간 (단위: 분).
    10. **vote_average**: TMDb에서 제공하는 영화의 평균 평점.
    11. **vote_count**: 영화에 대한 평가 개수.
    12. **production_companies**: 영화의 제작 회사 리스트.
    13. **production_countries**: 영화의 제작 국가 리스트.
    14. **spoken_languages**: 영화에서 사용된 언어 리스트.
    15. **cast**: 주요 출연진 리스트.
    16. **crew**: 영화 제작에 참여한 주요 제작진 리스트.
    17. **keywords**: 영화의 키워드 리스트.
    18. **tagline**: 영화의 태그라인(주요 홍보 문구).
    19. **original_language**: 영화의 원어 (예: 영어, 한국어 등).
    20. **homepage**: 영화의 공식 웹사이트 URL.
    21. **poster_path**: 영화 포스터 이미지 URL 경로.

In [2]:
import numpy as np
import pandas as pd

In [3]:

movie_df = pd.read_csv('./data/tmdb_5000_movies.csv')
movie_df.head()
movie_df.shape

(4803, 20)

In [4]:
movie_df = movie_df[['id', 'title', 'genres', 'vote_average', 'vote_count', 'popularity', 'keywords', 'overview']]

In [5]:
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            4803 non-null   int64  
 1   title         4803 non-null   object 
 2   genres        4803 non-null   object 
 3   vote_average  4803 non-null   float64
 4   vote_count    4803 non-null   int64  
 5   popularity    4803 non-null   float64
 6   keywords      4803 non-null   object 
 7   overview      4800 non-null   object 
dtypes: float64(2), int64(2), object(4)
memory usage: 300.3+ KB


In [6]:
from ast import literal_eval

movie_df['genres'] = movie_df['genres'].apply(literal_eval)


In [7]:

movie_df['genres'] = movie_df['genres'].apply(lambda genres : [genre['name'] for genre in genres])
movie_df['genres']

0       [Action, Adventure, Fantasy, Science Fiction]
1                        [Adventure, Fantasy, Action]
2                          [Action, Adventure, Crime]
3                    [Action, Crime, Drama, Thriller]
4                [Action, Adventure, Science Fiction]
                            ...                      
4798                        [Action, Crime, Thriller]
4799                                [Comedy, Romance]
4800               [Comedy, Drama, Romance, TV Movie]
4801                                               []
4802                                    [Documentary]
Name: genres, Length: 4803, dtype: object

In [8]:
movie_df['genres'] = movie_df['genres'].apply(lambda x : ' '.join(x))
movie_df['genres']

0       Action Adventure Fantasy Science Fiction
1                       Adventure Fantasy Action
2                         Action Adventure Crime
3                    Action Crime Drama Thriller
4               Action Adventure Science Fiction
                          ...                   
4798                       Action Crime Thriller
4799                              Comedy Romance
4800               Comedy Drama Romance TV Movie
4801                                            
4802                                 Documentary
Name: genres, Length: 4803, dtype: object

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(ngram_range=(1,2))
genres_vec  = count_vectorizer.fit_transform(movie_df['genres'])
print(genres_vec.toarray()[:5])
genres_vec_vocab = pd.DataFrame(count_vectorizer.get_feature_names_out())
genres_vec_vocab


[[1 1 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 1 0 ... 0 0 0]]


,0
0,action
1,action adventure
2,action animation
3,action comedy
4,action crime
...,...
271,western drama
272,western history
273,western music
274,western romance


### 코사인 유사도 측정


In [10]:
from sklearn.metrics.pairwise import cosine_similarity

genres_sim = cosine_similarity(genres_vec,genres_vec)
genres_sim.shape

(4803, 4803)

In [13]:
movie_idx_by_genres_sim = genres_sim.argsort(axis=1)[:,::-1]

In [21]:
def recommend_movie_by_genres(movie_title,top_n=10):
    movie = movie_df[movie_df['title'] == movie_title]
    if movie.empty:
        return '없는 영화 입니다.'

    movie_idx = movie.index
    movie_df['genres_sim'] = movie_df['genres'].reshape(-1)

    topn_movie_idx = movie_idx_by_genres_sim[movie_idx,:(top_n)*2]
    topn_movie_idx = topn_movie_idx.reshape(-1)
    return movie_df.iloc[topn_movie_idx].sort_values('genres_sim',ascending=False).head(top_n)

In [ ]:
### 평점을 반영한 추천 시스템

**가중평점** 

$$
가중 평점(Weighted Rating) = (v/(v+m)) * R + (m/(v+m)) * C
$$

-  v: 개별 영화에 평점을 투표한 횟수. vote_count
-  m: 평점을 부여하기 위한 최소 투표 횟수. 임계치 설정(직접)
-  R: 개별 영화에 대한 평균 평점. vote_average
-  C: 전체 영화에 대한 평균 평점. 전체적으로 평점이 후한 편인지, 박한 편인지 반영


In [16]:
movie_df[['title','vote_average','vote_count']].sort_values('vote_average',ascending=False).head()

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2


In [18]:
m = movie_df['vote_count'].quantile(0.6)
C = movie_df['vote_average'].mean()

def weighted_rating(movie):
    v = movie['vote_count']
    R = movie['vote_average']
    return (v/(v+m))*R + (m/(v+m))*C

movie_df['weighted_vote'] = movie_df.apply(weighted_rating,axis=1)
movie_df[['title','vote_average','vote_count','weighted_vote']].sort_values('weighted_vote',ascending=False).head(10)



,title,vote_average,vote_count,weighted_vote
1881,The Shawshank Redemption,8.5,8205,8.396052
3337,The Godfather,8.4,5893,8.263591
662,Fight Club,8.3,9413,8.216455
3232,Pulp Fiction,8.3,8428,8.207102
65,The Dark Knight,8.2,12002,8.136930
1818,Schindler's List,8.3,4329,8.126069
3865,Whiplash,8.3,4254,8.123248
809,Forrest Gump,8.2,7927,8.105954
2294,Spirited Away,8.3,3840,8.105867
2731,The Godfather: Part II,8.3,3338,8.079586
